## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [2]:
%%sql
postgresql:///animal_shelters

'Connected: @animal_shelters'

In [3]:
%sql select animalid, DATE_PART('year', AGE('2021-12-31', TO_DATE(birthdate,'MM/DD/YYYY'))) AS age, animaltype, weight, location, case when animaltype='Dog' and weight<=10 then 'Small' when animaltype='Dog' and weight between 11 and 30 then 'Medium' when animaltype='Dog' and weight>30 then 'Large' when animaltype='Cat' and weight<=5 then 'Small' when animaltype='Cat' and weight between 6 and 7 then 'Medium' when animaltype='Cat' and weight>7 then 'Large' when animaltype='Bird' and weight<=0.7 then 'Small' when animaltype='Bird' and (weight>0.7 and weight<=1.1) then 'Medium' when animaltype='Bird' and weight>1.1 then 'Large' end AS size from animals limit 5;

 * postgresql:///animal_shelters
5 rows affected.


animalid,age,animaltype,weight,location,size
80066a4b41ac06,2.0,Cat,4.0,Fort Worth,Small
968bf2e7230d2f,9.0,Cat,7.0,Houston,Medium
8aacfe19c56b0f,8.0,Dog,45.0,Austin,Large
7126d96889803f,6.0,Cat,3.0,Houston,Small
9617240ad5b268,7.0,Cat,9.0,Houston,Large


In [4]:
%sql CREATE TABLE newanimal AS (select animalid, DATE_PART('year', AGE('2021-12-31', TO_DATE(birthdate,'MM/DD/YYYY'))) AS age, animaltype, weight, location, case when animaltype='Dog' and weight<=10 then 'Small' when animaltype='Dog' and weight between 11 and 30 then 'Medium' when animaltype='Dog' and weight>30 then 'Large' when animaltype='Cat' and weight<=5 then 'Small' when animaltype='Cat' and weight between 6 and 7 then 'Medium' when animaltype='Cat' and weight>7 then 'Large' when animaltype='Bird' and weight<=0.7 then 'Small' when animaltype='Bird' and (weight>0.7 and weight<=1.1) then 'Medium' when animaltype='Bird' and weight>1.1 then 'Large' end AS size from animals);

 * postgresql:///animal_shelters
11899 rows affected.


[]

In [5]:
%sql select * from newanimal limit 5;

 * postgresql:///animal_shelters
5 rows affected.


animalid,age,animaltype,weight,location,size
80066a4b41ac06,2.0,Cat,4.0,Fort Worth,Small
968bf2e7230d2f,9.0,Cat,7.0,Houston,Medium
8aacfe19c56b0f,8.0,Dog,45.0,Austin,Large
7126d96889803f,6.0,Cat,3.0,Houston,Small
9617240ad5b268,7.0,Cat,9.0,Houston,Large


In [6]:
%sql select * from newanimal where animalid not in (select sponsorid from sponsored_animals) limit 5;

 * postgresql:///animal_shelters
5 rows affected.


animalid,age,animaltype,weight,location,size
968bf2e7230d2f,9.0,Cat,7.0,Houston,Medium
8aacfe19c56b0f,8.0,Dog,45.0,Austin,Large
9617240ad5b268,7.0,Cat,9.0,Houston,Large
fd7bf1fa986957,4.0,Dog,39.0,Austin,Large
691081c83b1075,5.0,Dog,3.0,Fort Worth,Small


In [7]:
%sql CREATE TABLE notsponsored AS (select * from newanimal where animalid not in (select sponsorid from sponsored_animals));

 * postgresql:///animal_shelters
9757 rows affected.


[]

In [8]:
%sql select age,animaltype, weight,size,costs from notsponsored as a inner join location_costs as b on a.location=b.location limit 5;

 * postgresql:///animal_shelters
5 rows affected.


age,animaltype,weight,size,costs
9.0,Cat,7.0,Medium,140
8.0,Dog,45.0,Large,135
7.0,Cat,9.0,Large,140
4.0,Dog,39.0,Large,135
5.0,Dog,3.0,Small,100


In [9]:
%sql CREATE TABLE loc_cost AS (select age,animaltype, weight,size,costs from notsponsored as a inner join location_costs as b on a.location=b.location);

 * postgresql:///animal_shelters
9757 rows affected.


[]

In [10]:
%sql select * from loc_cost limit 5;

 * postgresql:///animal_shelters
5 rows affected.


age,animaltype,weight,size,costs
9.0,Cat,7.0,Medium,140
8.0,Dog,45.0,Large,135
7.0,Cat,9.0,Large,140
4.0,Dog,39.0,Large,135
5.0,Dog,3.0,Small,100


In [11]:
%sql CREATE TABLE age_cost AS (select CAST(age AS decimal), costs from age_costs);

 * postgresql:///animal_shelters
51 rows affected.


[]

In [12]:
%sql SELECT a.animaltype, a.size, a.costs, b.costs FROM loc_cost as a inner join age_cost as b on a.age=b.age limit 5;

 * postgresql:///animal_shelters
5 rows affected.


animaltype,size,costs,costs_1
Cat,Medium,140,150
Dog,Large,135,150
Cat,Large,140,100
Dog,Large,135,0
Dog,Small,100,100


In [13]:
%sql select * from size_costs limit 5;

 * postgresql:///animal_shelters
5 rows affected.


sizeid,animaltype,size,costs
DL,Dog,Large,175
DM,Dog,Medium,150
DS,Dog,Small,125
CL,Cat,Large,140
CM,Cat,Medium,120


In [14]:
%sql SELECT * FROM loc_cost as a inner join age_cost as b on a.age=b.age inner join size_costs as c on a.animaltype=c.animaltype and a.size=c.size LIMIT 5; 

 * postgresql:///animal_shelters
5 rows affected.


age,animaltype,weight,size,costs,age_1,costs_1,sizeid,animaltype_1,size_1,costs_2
0.0,Dog,14.0,Medium,135,0,0,DM,Dog,Medium,150
0.0,Cat,3.0,Small,140,0,0,CS,Cat,Small,100
0.0,Cat,5.0,Small,100,0,0,CS,Cat,Small,100
0.0,Cat,4.0,Small,100,0,0,CS,Cat,Small,100
0.0,Dog,49.0,Large,135,0,0,DL,Dog,Large,175


In [15]:
%sql SELECT a.animaltype, a.size, (a.costs + b.costs + c.costs) AS total FROM loc_cost as a inner join age_cost as b on a.age=b.age inner join size_costs as c on a.animaltype=c.animaltype and a.size=c.size LIMIT 5; 

 * postgresql:///animal_shelters
5 rows affected.


animaltype,size,total
Dog,Medium,285
Cat,Small,240
Cat,Small,200
Cat,Small,200
Dog,Large,310


In [16]:
%sql CREATE TABLE total AS (SELECT a.animaltype, a.size, (a.costs + b.costs + c.costs) AS total FROM loc_cost as a inner join age_cost as b on a.age=b.age inner join size_costs as c on a.animaltype=c.animaltype and a.size=c.size);

 * postgresql:///animal_shelters
9757 rows affected.


[]

In [17]:
%sql SELECT * FROM total limit 10;

 * postgresql:///animal_shelters
10 rows affected.


animaltype,size,total
Dog,Medium,290
Dog,Medium,270
Cat,Small,220
Cat,Large,250
Cat,Small,235
Cat,Large,280
Cat,Small,240
Dog,Medium,290
Cat,Small,235
Dog,Large,310


In [18]:
%sql SELECT animaltype, size, SUM(total) FROM total GROUP BY animaltype, size;

 * postgresql:///animal_shelters
9 rows affected.


animaltype,size,sum
Cat,Large,439490
Dog,Large,977665
Bird,Large,8120
Dog,Medium,941895
Cat,Medium,250575
Bird,Small,1615
Cat,Small,518015
Bird,Medium,3130
Dog,Small,336530


In [19]:
%sql CREATE TABLE summary AS (SELECT animaltype, size, SUM(total) FROM total GROUP BY animaltype, size);

 * postgresql:///animal_shelters
9 rows affected.


[]

In [20]:
%sql SELECT * FROM summary ORDER BY animaltype ASC, CASE WHEN size='Small' THEN 1 WHEN size='Medium' THEN 2 WHEN size='Large' THEN 3 END;

 * postgresql:///animal_shelters
9 rows affected.


animaltype,size,sum
Bird,Small,1615
Bird,Medium,3130
Bird,Large,8120
Cat,Small,518015
Cat,Medium,250575
Cat,Large,439490
Dog,Small,336530
Dog,Medium,941895
Dog,Large,977665


In [21]:
%sql CREATE TABLE ordered AS (SELECT * FROM summary ORDER BY animaltype ASC, CASE WHEN size='Small' THEN 1 WHEN size='Medium' THEN 2 WHEN size='Large' THEN 3 END);

 * postgresql:///animal_shelters
9 rows affected.


[]

In [22]:
%sql select animaltype, size, sum AS total, ROUND((sum/(SELECT SUM(sum) from ordered)*100),2) AS percentage from ordered;

 * postgresql:///animal_shelters
9 rows affected.


animaltype,size,total,percentage
Bird,Small,1615,0.05
Bird,Medium,3130,0.09
Bird,Large,8120,0.23
Cat,Small,518015,14.90
Cat,Medium,250575,7.21
Cat,Large,439490,12.64
Dog,Small,336530,9.68
Dog,Medium,941895,27.09
Dog,Large,977665,28.12
